In [20]:
# %%
import pandas as pd

# Wczytanie danych z pliku CSV
df = pd.read_csv(r'afteretl.csv')

# %%
df.head()



,Unnamed: 0,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,...,drug substances,annual earnings,value_divided,Time,Month,Day,Year,Quarter,Day of Week,Vict Age bucket
0,0,10304468,2020-01-08,01/08/2020,2230,3,Southwest,377,2,624,...,Alcohol,118908,22.3,22,1,8,2020,1,Wednesday,35-60
1,1000,200105946,2020-01-29,01/29/2020,720,1,Central,157,2,624,...,Prescription opioids,27044,7.2,7,1,29,2020,1,Wednesday,35-60
2,2000,200205282,2020-01-27,01/27/2020,810,2,Rampart,249,2,956,...,Heroin,127062,8.1,8,1,27,2020,1,Monday,35-60
3,3000,200110265,2020-04-09,03/31/2020,1930,1,Central,163,1,350,...,Cannabis (Marijuana),144944,19.3,19,3,31,2020,1,Thursday,35-60
4,4000,200404736,2020-01-20,01/19/2020,2030,4,Hollenbeck,437,2,627,...,Alcohol,101813,20.3,20,1,19,2020,1,Monday,0-18


In [21]:
df.columns

Index(['Unnamed: 0', 'DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA',
       'AREA NAME', 'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc',
       'Mocodes', 'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd',
       'Premis Desc', 'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc',
       'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION',
       'Cross Street', 'LAT', 'LON', 'gender of criminal',
       'country of criminal', 'kill count', 'marital status of criminal ',
       'age of criminal', 'previously punished', 'education levels',
       'drug substances', 'annual earnings', 'value_divided', 'Time', 'Month',
       'Day', 'Year', 'Quarter', 'Day of Week', 'Vict Age bucket'],
      dtype='object')

In [22]:
df['count'] = 1
df = df.groupby(
    ["Year", "Quarter", "Month", "Day", "AREA NAME", "gender of criminal", "age of criminal", 'country of criminal']
).agg({'kill count': 'sum'}).reset_index()

# Zmieniamy nazwę kolumny 'kill count' na 'KillCount'
df.rename(columns={'kill count': 'KillCount'}, inplace=True)

In [17]:
df['age of criminal'].unique

<bound method Series.unique of 0       35-60
1       18-35
2       18-35
3       35-60
4       18-35
        ...  
661     35-60
662    60-100
663     18-35
664     35-60
665      0-18
Name: age of criminal, Length: 666, dtype: object>

In [23]:
df

,Year,Quarter,Month,Day,AREA NAME,gender of criminal,age of criminal,country of criminal,KillCount
0,2020,1,1,2,Southeast,Male,35-60,Black,0
1,2020,1,1,3,Topanga,Female,18-35,Other,10
2,2020,1,1,3,Van Nuys,X-gender,18-35,Other,2
3,2020,1,1,4,Topanga,Male,35-60,Other,2
4,2020,1,1,5,Northeast,Male,18-35,Black,4
...,...,...,...,...,...,...,...,...,...
661,2023,3,7,3,Mission,Male,35-60,Black,0
662,2023,3,7,4,77th Street,Male,60-100,Black,4
663,2023,3,7,6,Olympic,Male,18-35,Other,6
664,2023,3,7,7,Northeast,Male,35-60,Black,10


In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go

# Tworzenie ramki danych


# Grupowanie i agregacja danych
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
df_grouped = df.groupby('Date')['KillCount'].sum().reset_index()

# Przygotowanie danych do modelu XGBoost
df_grouped['Month'] = df_grouped['Date'].dt.month
df_grouped['Day'] = df_grouped['Date'].dt.day

X = df_grouped[['Month', 'Day']]
y = df_grouped['KillCount']

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Dopasowanie modelu XGBoost
model = xgb.XGBRegressor(objective ='reg:squarederror', seed=42)
model.fit(X_train, y_train)

# Prognoza na następny rok
next_year_dates = pd.date_range(start=df_grouped['Date'].max() + pd.DateOffset(days=1), end=df_grouped['Date'].max() + pd.DateOffset(years=1), freq='D')
next_year_X = pd.DataFrame({'Month': next_year_dates.month, 'Day': next_year_dates.day})
forecast = model.predict(next_year_X)

# Wykres szeregu czasowego z prognozą
fig = go.Figure()

# Dodanie rzeczywistych danych
fig.add_trace(go.Scatter(x=df_grouped['Date'], y=df_grouped['KillCount'], mode='lines+markers', name='Actual'))

# Dodanie prognoz
fig.add_trace(go.Scatter(x=next_year_dates, y=forecast, mode='lines+markers', name='Forecast', line=dict(color='red', dash='dash')))

# Aktualizacja układu wykresu
fig.update_layout(title='Liczba zabitych w przestępstwach w czasie z prognozą na następny rok', xaxis_title='Data', yaxis_title='Liczba zabitych',template="plotly_dark",)

# Wyświetlenie wykresu
fig.show()
# Zapisywanie wykresu do pliku HTML
fig.write_html("forecast_plot2_1.html")


In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import plotly.graph_objects as go



df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
# Grupowanie i agregacja danych
df_grouped = df.groupby('Date')['KillCount'].sum().reset_index()

# Przygotowanie danych do modelu ARIMA
df_grouped.set_index('Date', inplace=True)

# Dopasowanie modelu ARIMA
model = ARIMA(df_grouped['KillCount'], order=(5, 1, 0))
model_fit = model.fit()

# Prognoza na następny rok
forecast = model_fit.forecast(steps=365)
forecast_dates = pd.date_range(start=df_grouped.index[-1] + pd.DateOffset(days=1), periods=365, freq='D')

# Wykres szeregu czasowego z prognozą
fig = go.Figure()

# Dodanie rzeczywistych danych
fig.add_trace(go.Scatter(x=df_grouped.index, y=df_grouped['KillCount'], mode='lines+markers', name='Actual'))

# Dodanie prognoz
fig.add_trace(go.Scatter(x=forecast_dates, y=forecast, mode='lines+markers', name='Forecast', line=dict(color='red', dash='dash')))

# Aktualizacja układu wykresu
fig.update_layout(title='Liczba zabitych czasie z prognozą na następny rok za pomocą algorytmu ARIMA', xaxis_title='Data', yaxis_title='Liczba incydentów')

# Wyświetlenie wykresu
fig.show()

# Zapisywanie wykresu do pliku HTML
fig.write_html("forecast_plot2_2.html")


c:\Users\micha\Desktop\WAD_07\Wielowymiarowa_analiza_danych\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\micha\Desktop\WAD_07\Wielowymiarowa_analiza_danych\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\micha\Desktop\WAD_07\Wielowymiarowa_analiza_danych\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\micha\Desktop\WAD_07\Wielowymiarowa_analiza_danych\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an 

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
import plotly.graph_objects as go



df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])

# Grupowanie i agregacja danych
df_grouped = df.groupby('Date')['KillCount'].sum().reset_index()

# Przygotowanie danych do modelu Prophet
df_grouped.rename(columns={'Date': 'ds', 'KillCount': 'y'}, inplace=True)

# Dopasowanie modelu Prophet
model = Prophet()
model.fit(df_grouped)

# Prognoza na następny rok
future_dates = model.make_future_dataframe(periods=365)
forecast = model.predict(future_dates)

# Wykres szeregu czasowego z prognozą
fig = go.Figure()

# Dodanie rzeczywistych danych
fig.add_trace(go.Scatter(x=df_grouped['ds'], y=df_grouped['y'], mode='lines+markers', name='Actual'))

# Dodanie prognoz
forecast_future = forecast[forecast['ds'] > df_grouped['ds'].max()]
fig.add_trace(go.Scatter(x=forecast_future['ds'], y=forecast_future['yhat'], mode='lines+markers', name='Forecast', line=dict(color='red', dash='dash')))

# Aktualizacja układu wykresu
fig.update_layout(title='Liczba zabitych w czasie z prognozą na następny rok za pomocą algorytmu Prophet', xaxis_title='Data', yaxis_title='Liczba KillCount')

# Wyświetlenie wykresu
fig.show()

# Zapisywanie wykresu do pliku HTML
fig.write_html("forecast_plot2_3.html")


c:\Users\micha\Desktop\WAD_07\Wielowymiarowa_analiza_danych\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

14:19:19 - cmdstanpy - INFO - Chain [1] start processing
14:19:19 - cmdstanpy - INFO - Chain [1] done processing


In [32]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go

# Tworzenie kolumny daty
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])

# Grupowanie i agregacja danych
df_grouped = df.groupby('Date')['KillCount'].sum().reset_index()

# Przygotowanie danych do modelu Random Forest
df_grouped['Month'] = df_grouped['Date'].dt.month
df_grouped['Day'] = df_grouped['Date'].dt.day

X = df_grouped[['Month', 'Day']]
y = df_grouped['KillCount']

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Dopasowanie modelu Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prognoza na następny rok
next_year_dates = pd.date_range(start=df_grouped['Date'].max() + pd.DateOffset(days=1), end=df_grouped['Date'].max() + pd.DateOffset(years=1), freq='D')
next_year_X = pd.DataFrame({'Month': next_year_dates.month, 'Day': next_year_dates.day})
forecast = model.predict(next_year_X)

# Wykres szeregu czasowego z prognozą
fig = go.Figure()

# Dodanie rzeczywistych danych
fig.add_trace(go.Scatter(x=df_grouped['Date'], y=df_grouped['KillCount'], mode='lines+markers', name='Actual'))

# Dodanie prognoz
fig.add_trace(go.Scatter(x=next_year_dates, y=forecast, mode='lines+markers', name='Forecast', line=dict(color='red', dash='dash')))

# Aktualizacja układu wykresu
fig.update_layout(title='Liczba izabitych w czasie z prognozą na następny rok za pomocą modelu Random Forest', xaxis_title='Data', yaxis_title='Liczba KillCount')

# Wyświetlenie wykresu
fig.show()

# Zapisywanie wykresu do pliku HTML
fig.write_html("forecast_plot2_4.html")
